# Import Package, and Func.

In [10]:
import numpy as np
import matplotlib.pyplot as plt
#import qiskit.quantum_info as qi

#from qiskit import Aer
from qiskit.primitives import Estimator
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.mappers import ParityMapper, JordanWignerMapper, BravyiKitaevMapper
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer
from qiskit_nature import settings
from qiskit_algorithms.optimizers import  SPSA, COBYLA, L_BFGS_B
from qiskit_nature.second_q.circuit.library import UCCSD, HartreeFock
from qiskit.circuit.library import TwoLocal
from qiskit_algorithms.minimum_eigensolvers import NumPyMinimumEigensolver, VQE
import pickle


In [11]:
def reset_intermediate_info():
    global intermediate_info
    intermediate_info.clear()
    print(intermediate_info)
    intermediate_info = {
        'nfev ': [],
        'parameters ': [],
        'energy ': [],
        'stddev ': []
    }


def exact_solver(qubit_op, problem):
    sol = NumPyMinimumEigensolver().compute_minimum_eigenvalue(qubit_op)
    result = problem.interpret(sol)
    return result

def fermion_to_qubit(problem, second_q_op, mapper_name,  z2symmetry_reduction=None):
  if mapper_name == "JW":
    mapper = JordanWignerMapper()
  if mapper_name == "Pa":
    mapper = ParityMapper(num_particles=problem.num_particles)
  if mapper_name == "BK":
    mapper = BravyiKitaevMapper()

  qubit_op = mapper.map(second_q_op)

  if z2symmetry_reduction != None:
    mapper = problem.get_tapered_mapper(mapper)
    qubit_op = mapper.map(second_q_op)
  else :
    qubit_op = mapper.map(second_q_op)

  return qubit_op , mapper

def least_Energy(as_problem, molecule_name):
    as_num_particles = as_problem.num_particles
    as_num_spatial_orbitals = as_problem.num_spatial_orbitals
    as_fermionic_hamiltonian = as_problem.hamiltonian
    as_second_q_op = as_fermionic_hamiltonian.second_q_op()
    energy_arr = []
    ansatz_order = []
    opt_order=[]
    qubit_op, mapper = fermion_to_qubit(as_problem, as_second_q_op, "Pa", z2symmetry_reduction=None)
    as_init_state = HartreeFock(as_num_spatial_orbitals,as_num_particles,mapper)
    uccsd = UCCSD(as_num_spatial_orbitals,as_num_particles,mapper,initial_state=as_init_state, generalized=True)
    twolocal = TwoLocal(as_num_spatial_orbitals*2, ['ry', 'rz'], 'cz', initial_state=as_init_state)
    ansatzs=[uccsd,twolocal]
    iter = [250,1000]
    ansatzs_name=['UCCSD','Two local']
    
    opt_names = ['COBYLA','SPSA','L-BFGS-B'] 
    noiseless_estimator = Estimator()
    for i in range(2):
        ansatz = ansatzs[i]
        ansatz_name= ansatzs_name[i]
        MAX_ITER = iter[i]
        opt_arr = [COBYLA(),SPSA(),L_BFGS_B(max_evals_grouped=3)]
        for k in range(3):
            reset_intermediate_info()
            optimizer = opt_arr[k]
            opt_name = opt_names[k]
            filename = '{0}_{1}'.format(ansatz_name , opt_name)

            def callback (nfev , parameters , energy , stddev):
              intermediate_info ['nfev ']. append ( nfev )
              intermediate_info ['parameters ']. append ( parameters )
              intermediate_info ['energy ']. append ( energy )
              intermediate_info ['stddev ']. append ( stddev )
              with open(f"./LiCoO2_data_x078/{molecule_name}/{filename}.pickle", "wb") as fw:   #pickel 파일 저장 부분
                pickle.dump(intermediate_info, fw)
                
            vqe = VQE(noiseless_estimator, ansatz, optimizer, callback = callback)
            result = vqe.compute_minimum_eigenvalue(qubit_op)
            # electronic_structure_result = as_problem.interpret(result)

            core = as_problem.hamiltonian.constants['ActiveSpaceTransformer']
            repulsion = as_problem.hamiltonian.constants['nuclear_repulsion_energy']

            vqe_result = result.eigenvalue.real #+shift

            exact_energy = exact_solver(qubit_op, as_problem).total_energies[0].real

            # convergence_energy = intermediate_info['energy '][-1]
            dimer_energy = vqe_result+repulsion + core
            Co_Li_energy = dimer_energy

            energy_arr.append(Co_Li_energy)
            ansatz_order.append(ansatz_name)
            opt_order.append(opt_name)
            
            plt.plot(range(len(intermediate_info['energy '])), intermediate_info['energy ']+repulsion + core )
            plt.axhline(exact_energy, color = 'r', linewidth = 1, label = 'exact energy')
            plt.title('{0},{1}'.format(ansatz_name , opt_name))
            plt.xlabel('VQE Iterations')
            plt.ylabel('Energy')
            plt.grid()
            plt.legend()
            plt.savefig('{0},{1}.png'.format(ansatz_name , opt_name))
            plt.show()
    
    return energy_arr, ansatz_order, opt_order

In [12]:
intermediate_info = {
'nfev ': [],
'parameters ': [],
'energy ': [],
'stddev ': []
}


basis = 'sto3g'
settings.use_pauli_sum_op = False

C = 1.9220
L = 2.0946
theta = np.deg2rad(94.24)

Co = (0,0,0)
O_1 = (C,0,0)
O_2 = (C*np.cos(theta),C*np.sin(theta),0)
Li = (C+L*np.cos(np.pi-theta),-L*np.sin(np.pi-theta),0)

# Energy calc.

## 1. O-Li(near, bonding)

In [ ]:
molecule_name = 'LiCoO2'

O_Li_dimer_atoms = ["Co","O", "O", "Li"]
O_Li_dimer_coords = [Co,O_1,O_2 Li]
O_Li_dimer_charge = 0
O_Li_dimer_multiplicity = 2

O_Li_moleculeinfo = MoleculeInfo(O_Li_dimer_atoms, O_Li_dimer_coords, charge=O_Li_dimer_charge, multiplicity=O_Li_dimer_multiplicity)

energy_arr =[]
order=[]
driver = PySCFDriver.from_molecule(O_Li_moleculeinfo, basis=basis)
E_problem = driver.run()
num_spatial_orbitals = E_problem.num_spatial_orbitals
num_particles = E_problem.num_particles
as_transformer = ActiveSpaceTransformer((6,6), 12)
as_problem = as_transformer.transform(E_problem)
energy_arr, ansatz_order, opt_order = least_Energy(as_problem, molecule_name=molecule_name)

print(energy_arr)

for k in range(len(energy_arr)):
    e=energy_arr[k]
    if e == np.min(energy_arr):
        print('optimal_calc :')
        print('anstaz: ', ansatz_order[k])
        print('optimizer: ', opt_order[k])
        print('Energy: ', e)
        
O_Li_dimer_energy=e